In [2]:
import pandas as pd
from datasets import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

# importing dataset
file_path = 'Train_data.xlsx'
df = pd.read_excel(file_path)

# print("Dataset head = ",df.head())
data = df.to_dict(orient='records')
dataset = Dataset.from_list(data)
# print("Dataset = ",dataset)


#Load the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')  # You can use 't5-base' if you want a larger model
model = T5ForConditionalGeneration.from_pretrained('t5-small')


def preprocess_function(examples):
    # Prefix the prompt with "translate: " to help the model understand the task
    inputs = [f"translate: {prompt}" for prompt in examples['EnglishQuestion']]
    
    # Tokenize both the inputs and the labels (targets) using the new `text_target` argument
    model_inputs = tokenizer(inputs, max_length=512, padding='max_length', truncation=True, text_target=examples['SQLQuery'])
    
    # Handling the case where the label is padding
    model_inputs['labels'] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in model_inputs['labels']
    ]

    return model_inputs


#Apply the preprocessing function to your dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

#Define training arguments
training_args = TrainingArguments(
    output_dir='./results',                   # Output directory
    per_device_train_batch_size=8,            # Batch size per device during training
    per_device_eval_batch_size=8,             # Batch size for evaluation
    num_train_epochs=5,                       # Total number of training epochs
    logging_dir='./logs',                     # Directory for storing logs
    logging_steps=10,                         # Log every 10 steps
    save_steps=100,                           # Keep this for compatibility, but we'll use epoch
    eval_strategy="epoch",                    # Change evaluation strategy to 'epoch' (replaced 'evaluation_strategy')
    save_strategy="epoch",                    # Change save strategy to 'epoch'
    save_total_limit=2,                       # Limit the number of saved checkpoints
    load_best_model_at_end=True               # Load the best model after training
)


#Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,          # Training data
    eval_dataset=tokenized_dataset,           # You can split this for validation if required
)

#Fine-tune the model
trainer.train()

# Step 12: Save the fine-tuned model and tokenizer
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.